In [1]:
# Scale LSP rasters from MODIS EVI
# Current raster band values are (TIMESAT) index based 
# from this we change it to actual day of the year values
# for example the scaled raster image of SOS having pixel value of 5 means 
# the start of the growing year occured on January 5

import numpy as np
import pandas as pd
import rioxarray as rxr
import rasterio as rio
import xarray as xr
import matplotlib.pyplot as plt
import os
import glob

In [29]:
# base variables
unscaled_raster_path = r"..\Data\Unscaled_LSP_Rasters"
scaled_raster_path  = r"..\Data\Scaled_LSP_Rasters"
start_year = 2001
end_year = 2024

In [30]:
#configure for Start of Season

LSP_metric = "sos"
var_path = os.path.join(unscaled_raster_path, LSP_metric)
var_files = sorted(glob.glob(os.path.join(var_path, "*.tif")))
var_dates = [pd.to_datetime(("20"+ os.path.basename(f)[-6:-4] ), format='%Y') for f in var_files]

var_raster_list = []
for f in var_files:
    r = rxr.open_rasterio(f, chunks={'x': 1024, 'y': 1024}, masked=True)
    var_raster_list.append(r)

var_rasters = xr.concat(var_raster_list, dim=pd.Index(var_dates, name='time'), join='exact')

del var_path, var_files, var_dates, var_raster_list 

In [31]:
#Scale SOS Rasters
for year in range(start_year, end_year+1):
    i = (year - start_year) + 2     #index must start from 2 since 2000 (dummy year) is technically the first 
    year_dt = pd.to_datetime(str(year), format='%Y')
    file_to_scale = var_rasters.sel(time=year_dt)

    
    valid_data = file_to_scale.where(file_to_scale > 0)
    
    
    int_part = np.floor(valid_data)
    dec_part = valid_data - int_part

    int_scaled = int_part - (23*i)
    int_scaled2 = 1 + ((int_scaled - 1) * 16)

    dec_scaled = dec_part * 16
    scaled_value = int_scaled2 + dec_scaled

    
    scaled_data = scaled_value.where(file_to_scale > 0, -999) 
    os.makedirs(os.path.join(scaled_raster_path, f"{LSP_metric}"), exist_ok=True)
    scaled_data.rio.to_raster(os.path.join(scaled_raster_path, f"{LSP_metric}", f"{LSP_metric}{year}.tif"))

del file_to_scale, valid_data, int_part, dec_part, int_scaled, int_scaled2, dec_scaled, scaled_value, scaled_data


In [32]:
#same workflow for End of Season

LSP_metric = "eos"
var_path = os.path.join(unscaled_raster_path, LSP_metric)
var_files = sorted(glob.glob(os.path.join(var_path, "*.tif")))
var_dates = [pd.to_datetime(("20"+ os.path.basename(f)[-6:-4] ), format='%Y') for f in var_files]

#load first raster to match for eos (2005 file is recovered from a different step therefore has slightly different dimensions)
match_raster = rxr.open_rasterio(os.path.join(scaled_raster_path, "sos", "sos2001.tif"))

var_raster_list = []
for f in var_files:
    r = rxr.open_rasterio(f, chunks={'x': 1024, 'y': 1024}, masked=True)
    r_match = r.rio.reproject_match(match_raster)
    var_raster_list.append(r_match)

var_rasters = xr.concat(var_raster_list, dim=pd.Index(var_dates, name='time'), join='exact')

for year in range(start_year, end_year + 1):
    i = (year - start_year) + 2   # (end of the season is occurs next year (365 + doy) therefore extra 1 is added)
    year_dt = pd.to_datetime(str(year), format='%Y')
    file_to_scale = var_rasters.sel(time=year_dt)

    
    valid_data = file_to_scale.where(file_to_scale > 0)
    
    
    int_part = np.floor(valid_data)
    dec_part = valid_data - int_part

    int_scaled = int_part - (23*i)
    int_scaled2 = 1 + ((int_scaled - 1) * 16)

    dec_scaled = dec_part * 16
    scaled_value = int_scaled2 + dec_scaled

    
    scaled_data = scaled_value.where(file_to_scale > 0, -999) 
    os.makedirs(os.path.join(scaled_raster_path, f"{LSP_metric}"), exist_ok=True)
    scaled_data.rio.to_raster(os.path.join(scaled_raster_path, f"{LSP_metric}", f"{LSP_metric}{year}.tif"))


del var_path, var_files, var_dates, var_raster_list, match_raster
del file_to_scale, valid_data, int_part, dec_part, int_scaled, int_scaled2, dec_scaled, scaled_value, scaled_data, r_match

In [33]:
#similar workflow for End of Season with revised scaling

LSP_metric = "los"
var_path = os.path.join(unscaled_raster_path, LSP_metric)
var_files = sorted(glob.glob(os.path.join(var_path, "*.tif")))
var_dates = [pd.to_datetime(("20"+ os.path.basename(f)[-6:-4] ), format='%Y') for f in var_files]

var_raster_list = []
for f in var_files:
    r = rxr.open_rasterio(f, chunks={'x': 1024, 'y': 1024}, masked=True)
    var_raster_list.append(r)

var_rasters = xr.concat(var_raster_list, dim=pd.Index(var_dates, name='time'), join='exact')

for year in range(start_year, end_year + 1):
    year_dt = pd.to_datetime(str(year), format='%Y')
    file_to_scale = var_rasters.sel(time=year_dt)

    #for los we simply take value and multiply by 16
    valid_data = file_to_scale.where(file_to_scale > 0)
    scaled_value = valid_data * 16    
    scaled_data = scaled_value.where(file_to_scale > 0, -999) 
    os.makedirs(os.path.join(scaled_raster_path, f"{LSP_metric}"), exist_ok=True)
    scaled_data.rio.to_raster(os.path.join(scaled_raster_path, f"{LSP_metric}", f"{LSP_metric}{year}.tif"))


del var_path, var_files, var_dates, var_raster_list
del file_to_scale, valid_data, scaled_value, scaled_data

In [34]:
#for pos scaling is not required but we put both no data values (-1 and -2) to -999

LSP_metric = "pos"
var_path = os.path.join(unscaled_raster_path, LSP_metric)
var_files = sorted(glob.glob(os.path.join(var_path, "*.tif")))
var_dates = [pd.to_datetime(("20"+ os.path.basename(f)[-6:-4] ), format='%Y') for f in var_files]

var_raster_list = []
for f in var_files:
    r = rxr.open_rasterio(f, chunks={'x': 1024, 'y': 1024}, masked=True)
    var_raster_list.append(r)

var_rasters = xr.concat(var_raster_list, dim=pd.Index(var_dates, name='time'), join='exact')

for year in range(start_year, end_year + 1):

    year_dt = pd.to_datetime(str(year), format='%Y')
    file_to_scale = var_rasters.sel(time=year_dt)

    valid_data = file_to_scale.where(file_to_scale > 0)
    scaled_value = valid_data
    scaled_data = scaled_value.where(file_to_scale > 0, -999) 
    os.makedirs(os.path.join(scaled_raster_path, f"{LSP_metric}"), exist_ok=True)
    scaled_data.rio.to_raster(os.path.join(scaled_raster_path, f"{LSP_metric}", f"{LSP_metric}{year}.tif"))


del var_path, var_files, var_dates, var_raster_list
del file_to_scale, valid_data, scaled_value, scaled_data

In [22]:
sos04 = rxr.open_rasterio(r"C:\Users\A S U S\Documents\Study\Land Surface Phenology\LSP_WS\Data\Scaled_LSP_Rasters\sos\sos2004.tif")
# Filter values between 0 and 1
values_0_1_mask = (sos04.values >= 323) 
values_0_1 = sos04.values[values_0_1_mask]
# Calculate statistics
count_0_1 = np.sum(values_0_1_mask)
# max_0_1 = np.max(values_0_1)
# mean_0_1 = np.mean(values_0_1)
# min_0_1 = np.min(values_0_1)
# print(f"\nValues between 0 and 1:")
print(f"Count: {count_0_1}")
# print(f"Max: {max_0_1:.6f}")
# print(f"Mean: {mean_0_1:.6f}")
# print(f"Min: {min_0_1:.6f}")

Count: 34
